In [4]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
import os
%matplotlib inline

In [5]:
def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    return cv2.addWeighted(initial_img, α, img, β, γ)

def process(image):
    gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    kernal_size = 5
    #kernal_size is always in odd numbers
    blur_gray = cv2.GaussianBlur(gray,(kernal_size,kernal_size),0)
    #plt.imshow(blur_gray, cmap = 'gray')
    low_threshold = 10
    high_threshold = 100
    edges = cv2.Canny(blur_gray,low_threshold,high_threshold)
    #plt.imshow(edges, cmap = 'gray')
    mask = np.zeros_like(edges)
    ignore_mask_color = 255
    imshape = image.shape
    vertices = np.array([[(0,imshape[0]),(450,320),(480,320),(imshape[1],imshape[0])]],dtype=np.int32)
    cv2.fillPoly(mask,vertices,ignore_mask_color)
    masked_edges = cv2.bitwise_and(edges,mask)
    #plt.imshow(masked_edges)
    """Hough transform"""
    rho=2
    theta = np.pi/180
    threshold = 15
    min_line_length = 40
    max_line_gap = 20
    line_image = np.copy(image)*0
    lines = cv2.HoughLinesP(masked_edges,rho,theta,threshold,np.array([]),min_line_length,max_line_gap)
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),10)
            
    color_edges = np.dstack((masked_edges,masked_edges,masked_edges))
    #plt.imshow(color_edges)
    line_edges = cv2.addWeighted(color_edges,0.8,line_image,1,0)
    #plt.imshow(line_edges)
    weightedImage = weighted_img(line_edges,image,α=0.8, β=1., γ=0.)
    result = weightedImage
    #plt.imshow(weightedImage)
    return result


In [14]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import time
import timeit

In [20]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

t:   0%|                                                                             | 0/125 [00:00<?, ?it/s, now=None]

Moviepy - Building video test_videos_output/solidWhiteRight.mp4.
Moviepy - Writing video test_videos_output/solidWhiteRight.mp4



Moviepy - Done !
Moviepy - video ready test_videos_output/solidWhiteRight.mp4
Wall time: 3.62 s


In [21]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))